# RAG Search Examples

This notebook demonstrates how to search the indexed medical instructions using semantic search.


In [1]:
import os
from pathlib import Path
import chromadb
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

load_dotenv()

CHROMA_DIR = Path(os.getenv("CHROMA_DIR", "storage/chroma"))
CHROMA_RAG_COLLECTION = os.getenv("CHROMA_RAG_COLLECTION", "instruction_chunks")
CHROMA_MEDICINES_COLLECTION = os.getenv("CHROMA_MEDICINES_COLLECTION", "medicines")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_EMBED_MODEL = os.getenv("OPENAI_EMBED_MODEL", "text-embedding-3-small")

# Initialize clients
chroma_client = chromadb.PersistentClient(path=str(CHROMA_DIR))
openai_client = OpenAI(api_key=OPENAI_API_KEY) if OPENAI_API_KEY else None

print(f"ChromaDB location: {CHROMA_DIR}")
print(f"RAG Collection: {CHROMA_RAG_COLLECTION}")


ChromaDB location: storage/chroma
RAG Collection: instruction_chunks


## Check RAG Collection Status


In [2]:
try:
    rag_collection = chroma_client.get_collection(CHROMA_RAG_COLLECTION)
    total_chunks = rag_collection.count()
    print(f"✅ RAG collection found: {total_chunks:,} chunks indexed")
    
    # Get sample to check structure
    sample = rag_collection.get(limit=1)
    if sample['ids']:
        print(f"\nSample chunk metadata:")
        print(sample['metadatas'][0] if sample['metadatas'] else "No metadata")
except Exception as e:
    print(f"❌ RAG collection not found: {e}")
    print("Please run indexing first: python -m app.indexing")


✅ RAG collection found: 1,203 chunks indexed

Sample chunk metadata:
{'chunk_index': 0, 'file_type': 'html', 'medicine_id': '4B54FAC4830861F2C22584B6002A242C', 'source_file': 'data/html/4B54FAC4830861F2C22584B6002A242C.html', 'total_chunks': 3}


## Semantic Search Function


In [4]:
def search_instructions(query: str, n_results: int = 5) -> list[dict]:
    """Search medical instructions using semantic search"""
    if not openai_client:
        print("❌ OpenAI API key not set")
        return []
    
    # Generate embedding for query
    response = openai_client.embeddings.create(
        model=OPENAI_EMBED_MODEL,
        input=query
    )
    query_embedding = response.data[0].embedding
    
    # Search in ChromaDB
    results = rag_collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )
    
    # Format results
    formatted_results = []
    for i in range(len(results['ids'][0])):
        formatted_results.append({
            'chunk_id': results['ids'][0][i],
            'document': results['documents'][0][i],
            'metadata': results['metadatas'][0][i],
            'distance': results['distances'][0][i] if results.get('distances') else None,
        })
    
    return formatted_results


## Example Searches


In [5]:
# Example 1: Search for side effects
query = "Які побічні ефекти?"  # "What are the side effects?" in Ukrainian
results = search_instructions(query, n_results=3)

print(f"Query: {query}")
print(f"Found {len(results)} results:\n")

for i, result in enumerate(results, 1):
    print(f"{'='*60}")
    print(f"Result {i} (distance: {result['distance']:.4f})")
    print(f"{'='*60}")
    print(f"Medicine ID: {result['metadata'].get('medicine_id', 'N/A')}")
    print(f"Source file: {result['metadata'].get('source_file', 'N/A')}")
    print(f"File type: {result['metadata'].get('file_type', 'N/A')}")
    print(f"Chunk {result['metadata'].get('chunk_index', 'N/A')} of {result['metadata'].get('total_chunks', 'N/A')}")
    print(f"\nContent preview:")
    print(result['document'][:300] + "..." if len(result['document']) > 300 else result['document'])
    print()


Query: Які побічні ефекти?
Found 3 results:

Result 1 (distance: 0.8308)
Medicine ID: 7853AE7986C81731C2258BE9002773D7
Source file: data/mht/UA178230101_7853.mht
File type: mht
Chunk 35 of 41

Content preview:
біль, вагініт; нечасто — висипання, диспепсія, метеоризм, блювання, зміни у випорожненнях, анорексія, закреп, запаморочення, сухість у роті, астенія, безсоння, лейкорея, кандидоз, свербіж, сонливість. Про нижченаведені побічні реакції повідомлялось при застосуванні антибіотиків класу цефалоспоринів:...

Result 2 (distance: 0.8645)
Medicine ID: D4158EF86161F3A6C2258BDA004975B7
Source file: data/mht/UA178490101_D415.mht
File type: mht
Chunk 26 of 31

Content preview:
ксичні ознаки і симптоми після передозування іншими β-лактамними антибіотиками включали нудоту, блювання, дискомфорт в епігастрії, діарею і судоми. Цефдінір видаляється з організму шляхом гемодіалізу. Цю інформацію слід враховувати у випадку серйозної інтоксикації внаслідок передозування, особливо п...

Result 3 (dista

In [6]:
# Example 2: Search for dosage information
query = "Як приймати ліки? Дозування"  # "How to take medicine? Dosage" in Ukrainian
results = search_instructions(query, n_results=3)

print(f"Query: {query}")
print(f"Found {len(results)} results:\n")

for i, result in enumerate(results, 1):
    print(f"{'='*60}")
    print(f"Result {i}")
    print(f"{'='*60}")
    print(f"Medicine ID: {result['metadata'].get('medicine_id', 'N/A')}")
    print(f"Content: {result['document'][:400]}...")
    print()


Query: Як приймати ліки? Дозування
Found 3 results:

Result 1
Medicine ID: 7853AE7986C81731C2258BE9002773D7
Content: ь керувати транспортними засобами та працювати з іншими механізмами. Спосіб застосування та дози. Лікарський засіб у формі капсул призначений для застосування дорослим та підліткам віком від 13 років. Рекомендовані дози і тривалість лікування інфекцій у дорослих та підлітків наведені в таблиці нижче. Загальна добова доза при всіх інфекціях становить 600 мг. Прийом 1 раз на добу протягом 10 днів на...

Result 2
Medicine ID: 7E1AA6328B8246A5C2258C9100216286
Content: комендованій дозі погіршує увагу, швидкість реакції або здатність керувати транспортними засобами. Однак деякі пацієнти можуть відчувати сонливість, втому та астенію під час лікування левоцетиризином. Тому пацієнти, які мають намір керувати транспортними засобами, займатися потенційно небезпечною діяльністю або працювати з механізмами, повинні враховувати свою реакцію на лікарський засіб. Спосіб з...

Result 3


## Multilingual RAG: Ukrainian Instructions → English Responses

GPT-4o-mini is multilingual and can:
- Understand Ukrainian instructions from the database
- Respond in English (or any other language you specify)
- Translate and explain medical information across languages

This is perfect for international users who want to understand Ukrainian medical instructions in their own language.


In [ ]:
OPENAI_LLM_MODEL = os.getenv("OPENAI_LLM_MODEL", "gpt-4o-mini")

def ask_rag_question(
    query: str,
    response_language: str = "English",
    n_results: int = 3,
    max_context_chars: int = 2000
) -> dict:
    """
    Ask a question in any language, get response in specified language.
    
    Args:
        query: Question in any language (e.g., Ukrainian)
        response_language: Language for the response (e.g., "English", "Ukrainian", "Russian")
        n_results: Number of relevant chunks to retrieve
        max_context_chars: Maximum characters of context to include
    
    Returns:
        dict with 'answer', 'sources', 'chunks_used'
    """
    if not openai_client:
        return {"error": "OpenAI API key not set"}
    
    # Step 1: Semantic search to find relevant chunks
    search_results = search_instructions(query, n_results=n_results)
    
    if not search_results:
        return {"error": "No relevant information found"}
    
    # Step 2: Build context from search results
    context_parts = []
    sources = []
    
    for i, result in enumerate(search_results, 1):
        chunk_text = result['document']
        medicine_id = result['metadata'].get('medicine_id', 'N/A')
        source_file = result['metadata'].get('source_file', 'N/A')
        
        # Truncate if too long
        if len(chunk_text) > max_context_chars // n_results:
            chunk_text = chunk_text[:max_context_chars // n_results] + "..."
        
        context_parts.append(f"[Source {i} - Medicine ID: {medicine_id}]\n{chunk_text}")
        sources.append({
            'medicine_id': medicine_id,
            'source_file': source_file,
            'chunk_index': result['metadata'].get('chunk_index', 'N/A'),
        })
    
    context = "\n\n".join(context_parts)
    
    # Step 3: Build prompt for LLM
    system_prompt = f"""You are a medical information assistant. You help users understand medical instructions.
The medical instructions are in Ukrainian, but you should respond in {response_language}.

Your task:
1. Understand the Ukrainian medical instruction content provided
2. Answer the user's question based on the provided context
3. Respond clearly and accurately in {response_language}
4. If the context doesn't contain enough information, say so
5. Always cite which source(s) you used (Source 1, Source 2, etc.)

Be professional, accurate, and helpful."""

    user_prompt = f"""Question: {query}

Relevant medical instruction context (in Ukrainian):
{context}

Please answer the question in {response_language} based on the provided context."""

    # Step 4: Get response from LLM
    try:
        response = openai_client.chat.completions.create(
            model=OPENAI_LLM_MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,  # Lower temperature for more factual responses
        )
        
        answer = response.choices[0].message.content
        
        return {
            "answer": answer,
            "sources": sources,
            "chunks_used": len(search_results),
            "model": OPENAI_LLM_MODEL,
            "tokens_used": response.usage.total_tokens if hasattr(response, 'usage') else None,
        }
    except Exception as e:
        return {"error": f"Error generating response: {e}"}

print("✅ Multilingual RAG function ready!")
print(f"LLM Model: {OPENAI_LLM_MODEL}")


### Example 1: Ukrainian Question → English Answer


In [ ]:
# Ask a question in Ukrainian, get answer in English
ukrainian_question = "Які побічні ефекти цього препарату?"  # "What are the side effects of this medicine?"

result = ask_rag_question(
    query=ukrainian_question,
    response_language="English",
    n_results=3
)

if "error" in result:
    print(f"❌ Error: {result['error']}")
else:
    print("=" * 70)
    print("QUESTION (Ukrainian):")
    print(f"  {ukrainian_question}")
    print()
    print("ANSWER (English):")
    print(f"  {result['answer']}")
    print()
    print(f"Sources used: {result['chunks_used']} chunks")
    if result.get('tokens_used'):
        print(f"Tokens used: {result['tokens_used']:,}")
    print("=" * 70)
    
    # Show source details
    print("\nSource Details:")
    for i, source in enumerate(result['sources'], 1):
        medicine_info = get_medicine_info(source['medicine_id'])
        print(f"  Source {i}:")
        print(f"    Medicine: {medicine_info.get('ukrainian_name', 'N/A')}")
        print(f"    International name: {medicine_info.get('international_name', 'N/A')}")
        print(f"    File: {source['source_file']}")


### Example 2: Ukrainian Question → Ukrainian Answer (with translation option)


In [ ]:
# Ask about dosage in Ukrainian, get detailed answer in Ukrainian
ukrainian_question = "Як правильно приймати цей препарат? Яка доза?"

result = ask_rag_question(
    query=ukrainian_question,
    response_language="Ukrainian",  # Can also use "English", "Russian", etc.
    n_results=2
)

if "error" in result:
    print(f"❌ Error: {result['error']}")
else:
    print("=" * 70)
    print("QUESTION:")
    print(f"  {ukrainian_question}")
    print()
    print("ANSWER:")
    print(f"  {result['answer']}")
    print("=" * 70)


### Example 3: English Question → English Answer (works with Ukrainian content)


In [ ]:
# Even if you ask in English, it will find Ukrainian content and explain in English
english_question = "What are the contraindications for this medicine?"

result = ask_rag_question(
    query=english_question,
    response_language="English",
    n_results=3
)

if "error" in result:
    print(f"❌ Error: {result['error']}")
else:
    print("=" * 70)
    print("QUESTION (English):")
    print(f"  {english_question}")
    print()
    print("ANSWER (English, based on Ukrainian instructions):")
    print(f"  {result['answer']}")
    print()
    print(f"Model: {result['model']}")
    if result.get('tokens_used'):
        print(f"Tokens used: {result['tokens_used']:,}")
    print("=" * 70)


## Get Medicine Details for Search Results


In [7]:
def get_medicine_info(medicine_id: str) -> dict:
    """Get medicine information from medicines collection"""
    try:
        medicines_collection = chroma_client.get_collection(CHROMA_MEDICINES_COLLECTION)
        result = medicines_collection.get(ids=[medicine_id])
        
        if result['ids'] and result['metadatas']:
            return result['metadatas'][0]
        return {}
    except Exception as e:
        print(f"Error getting medicine info: {e}")
        return {}

# Example: Get medicine details for search results
query = "протипоказання"  # "contraindications" in Ukrainian
results = search_instructions(query, n_results=2)

print(f"Query: {query}\n")

for i, result in enumerate(results, 1):
    medicine_id = result['metadata'].get('medicine_id')
    medicine_info = get_medicine_info(medicine_id) if medicine_id else {}
    
    print(f"{'='*60}")
    print(f"Result {i}")
    print(f"{'='*60}")
    print(f"Medicine ID: {medicine_id}")
    print(f"Ukrainian name: {medicine_info.get('ukrainian_name', 'N/A')}")
    print(f"International name: {medicine_info.get('international_name', 'N/A')}")
    print(f"Source: {result['metadata'].get('source_file', 'N/A')}")
    print(f"\nRelevant chunk:")
    print(result['document'][:500] + "..." if len(result['document']) > 500 else result['document'])
    print()


Query: протипоказання

Result 1
Medicine ID: D3968618E3D6F00DC2258BB3002E79C6
Ukrainian name: L-ЛІЗИНУ ЕСЦИНАТ®
International name: Lysine
Source: data/mht/UA21310101_D396.mht

Relevant chunk:
Діти. Препарат протипоказаний дітям віком до 1 року. Передозування. Симптоми: відчуття жару, тахікардія, менорагія, нудота, печія, біль в епігастрії. Лікування: симптоматична терапія. Побічні реакції. При індивідуальній підвищеній чутливості до есцинату в окремих хворих можливі: алергічні реакції: шкірний висип (папульозний, петехіальний, еритематозний), свербіж, гіперемія шкіри, гіпертермія, кропив’янка, у поодиноких випадках – набряк Квінке, анафілактичний шок; з боку центральної і периферично...

Result 2
Medicine ID: 4F6429BC5B55C48CC2258CF1002533F0
Ukrainian name: 5-ФТОРУРАЦИЛ "ЕБЕВЕ
International name: Fluorouracil
Source: data/mht/UA60580101_4F64.mht

Relevant chunk:
ик небезпечної для життя або летальної побічної реакції і не повинні лікуватися 5-фторурацилом. Не було доведено, що рекомен

## Search and Display Results in DataFrame


In [8]:
# Search and create DataFrame
query = "алергічні реакції"  # "allergic reactions" in Ukrainian
results = search_instructions(query, n_results=5)

# Get medicine info for all results
medicines_collection = chroma_client.get_collection(CHROMA_MEDICINES_COLLECTION)
medicine_ids = [r['metadata'].get('medicine_id') for r in results if r['metadata'].get('medicine_id')]
medicine_data = medicines_collection.get(ids=medicine_ids) if medicine_ids else {'metadatas': []}

# Create DataFrame
data = []
for i, result in enumerate(results):
    medicine_id = result['metadata'].get('medicine_id', '')
    medicine_meta = {}
    if medicine_id in medicine_ids:
        idx = medicine_ids.index(medicine_id)
        if idx < len(medicine_data.get('metadatas', [])):
            medicine_meta = medicine_data['metadatas'][idx]
    
    data.append({
        'rank': i + 1,
        'medicine_id': medicine_id,
        'ukrainian_name': medicine_meta.get('ukrainian_name', 'N/A'),
        'international_name': medicine_meta.get('international_name', 'N/A'),
        'file_type': result['metadata'].get('file_type', 'N/A'),
        'chunk_index': result['metadata'].get('chunk_index', 'N/A'),
        'distance': result['distance'],
        'content_preview': result['document'][:150] + "..." if len(result['document']) > 150 else result['document'],
    })

df_results = pd.DataFrame(data)
print(f"Search results for: '{query}'")
display(df_results)


Search results for: 'алергічні реакції'


,rank,medicine_id,ukrainian_name,international_name,file_type,chunk_index,distance,content_preview
0,1,7853AE7986C81731C2258BE9002773D7,3-ДІНІР,Cefdinir,mht,38,0.939079,"рмальний некроліз, анафілактичні реакції, маку..."
1,2,8AFE938A35F68CD8C2258C9F0032A1C7,3-ДІНІР,Cefdinir,mht,4,0.977284,я протягом усього періоду клінічних вазомоторн...
2,3,E84CFE02A075231FC2258BF8003FEEAB,L-ТИРОКСИН 125 БЕРЛІН-ХЕМІ,Levothyroxine sodium,mht,32,0.981874,"нної системи: шок, анафілаксія (у рідкісних ви..."
3,4,DCED2B0101609E04C2258CBA002587C7,L-ТИРОКСИН 150 БЕРЛІН-ХЕМІ,Levothyroxine sodium,mht,35,1.040506,"ів (наприклад, ангіоневротичний набряк, шкірни..."
4,5,6C12C8A0C59FA873C2258CBA002571C9,АБ'ЮФЕН,Mono,mht,35,1.041343,"ів (наприклад, ангіоневротичний набряк, шкірни..."
